<a href="https://colab.research.google.com/github/DoItSon/Dacon/blob/main/%EA%B1%B4%EC%84%A4%EA%B8%B0%EA%B3%84%20%EC%98%A4%EC%9D%BC%20%EC%83%81%ED%83%9C%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D(tabnet)_%EC%97%B0%EC%8A%B5%ED%95%B4%EB%B3%B4%EA%B8%B0!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 메모장
- 0.8189379090413184 = 0.5
- 0.8236173245493615 = 0.4
- 0.8236625863676823 = 0.3
- 0.8256464235755943 = 0.35

- 0.823185924948775 머신러닝 결측치
- 0.8247486240372479 내 결측치

- 추후 베이지안 최저화를 통해 최적의 파라미터 찾기!

In [1]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 712 kB/s 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile

from pytorch_tabnet.tab_model  import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 32.3 MB/s 
     |████████████████████████████████| 81 kB 12.3 MB/s 
     |████████████████████████████████| 210 kB 57.4 MB/s 
     |████████████████████████████████| 78 kB 10.5 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 147 kB 73.2 MB/s 
     |████████████████████████████████| 112 kB 63.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=11ef3a0e67f54c16ab5079956be7a7290884e5ac49c08cd652920250273f7b0d
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


In [4]:
import optuna

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DATA_PATH = "/content/drive/MyDrive/"

In [7]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

In [8]:
categorical_features_enc = ['COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features_enc = ['ANONYMOUS_1', 'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B',
       'BA', 'BE', 'CA', 'CO', 'CR', 'CU', 'FE', 'H2O', 'LI', 'MG', 'MN', 'MO',
       'NA', 'NI', 'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'TI', 'V',
       'V40', 'ZN', 'CD', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FUEL', 'K', 'SOOTPERCENTAGE', 'U100', 'U75', 'U50', 'U25',
       'U20', 'U14', 'U6', 'U4', 'V100', 'COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']

## 전처리

In [9]:
CFG = {
    'n_splits':5,
    'SEED':42
}

- 머신러닝 결측치

In [10]:
cols = ['CD','FOPTIMETHGLY',"FUEL",'K','U100','U75','U50',"FH2O","FNOX","FOXID","FSO4","FTBN","SOOTPERCENTAGE","U25","U20","U14","U6","U4","V100"]

In [ ]:
for i in cols:
    imputer = IterativeImputer(estimator =  lgb.LGBMRegressor(),random_state=CFG["SEED"]) # IterativeImputer에서 다양한 옵션이 있다. (estimator는 머신러닝 옵션!)
    train[i] = imputer.fit_transform(train[[i]])

In [ ]:
train.isnull().sum().sum()

## 데이터 분리

In [ ]:
train_X  = train.drop(['ID', 'Y_LABEL'], axis = 1) # 정답값 제외
target  = train['Y_LABEL'] # 정답

test = test.drop(['ID'], axis = 1) # 추론 데이터

In [ ]:
train_X

# 원핫인코딩

In [ ]:
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_X[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
train_X = pd.concat([train_X,tmp],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

tmp1 = pd.DataFrame(
    enc.transform(test[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
test = pd.concat([test,tmp1],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

In [ ]:
train_X.shape,target.shape,test.shape

## 피처 추가

In [ ]:
hp = {'max_depth': 15,
 'min_samples_split': 2,
 'criterion': 'entropy',
 'max_leaf_nodes': 10,
 'n_estimators': 495,
 'learning_rate': 0.0943448646911196}

In [ ]:
cv = KFold(n_splits=5,shuffle=True,random_state=CFG["SEED"])

In [ ]:
score_list = []

model = LGBMClassifier(random_state=CFG['SEED'],**hp,use_missing=False) # use_missing=False 결측치 옵션
model.fit(train_X,target , eval_set = [(train_X,target)],early_stopping_rounds=50) 

pred = model.predict_proba(train_X)[:,1] # 1일 확률

# predict = np.where(pred > 0.35, 1 , 0)
score = cross_val_score(model,train_X,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
score_list.append(score) 

In [ ]:
x_target = pred
x_target

## Classifier

In [ ]:
train_X_toNum = train_X.to_numpy()

In [ ]:
target_toNum = target.to_numpy()

In [ ]:
test_toNum = test.to_numpy()

In [ ]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1)
        return score

In [ ]:
patience = 100
max_epochs = 2000
cv = KFold(n_splits=CFG['n_splits'], shuffle=True, random_state=CFG['SEED'])

In [ ]:
pred_list = []
score_list = []

model = TabNetClassifier(verbose = 1,seed = CFG['SEED'],optimizer_fn=torch.optim.AdamW)
model.fit(train_X_toNum,target_toNum ,
          eval_set = [(train_X_toNum,target_toNum)],
          patience=patience, max_epochs=max_epochs,
          eval_metric=['logloss','f1'])

pred = model.predict_proba(train_X_toNum)[:,1] 

score = cross_val_score(model,train_X_toNum,target_toNum,cv = cv ,scoring='f1_macro',n_jobs = -1)
score_list.append(score) 

In [ ]:
np.mean(score_list)
# 0.8405547354296713 tabnet
# 0.832199100324643
# 0.8402001798621617

- 확률 추출!

In [ ]:
x_target = pred
x_target

## Regressor

- train의 부분 X의 예측값을 설명하는 Regressor

In [ ]:
test_stage_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_COMPONENT1', 'COMPONENT_ARBITRARY_COMPONENT2',
       'COMPONENT_ARBITRARY_COMPONENT3', 'COMPONENT_ARBITRARY_COMPONENT4',
       'YEAR_2007', 'YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'YEAR_2018', 'YEAR_2019', 'YEAR_2020', 'YEAR_2021',
       'YEAR_2022']

In [ ]:
train_X = pd.DataFrame(train_X_toNum,columns = train_X.columns)
train_X

In [ ]:
train_reg = train_X[test_stage_features]
train_reg.shape,x_target.shape

In [ ]:
train_reg = train_reg.to_numpy()

In [ ]:
x_target = x_target.to_numpy()

In [ ]:
x_target = x_target.reshape(-1, 1)

In [ ]:
preds = []
score_list = []

model = TabNetRegressor(verbose = 1,seed = CFG['SEED'],optimizer_fn=torch.optim.AdamW,scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau)
model.fit(train_reg,x_target ,
          eval_set = [(train_reg,x_target)],
          patience=patience, max_epochs=max_epochs,
          eval_metric=['mae'])

score_list.append(model.best_cost)

pred = model.predict(train_reg)

In [ ]:
np.mean(score_list)
# 0.04416097
# 0.04255434 피처 추가
# 0.10485432
# 0.11118152

In [ ]:
pred = model.predict(test_toNum)

In [ ]:
pred

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit.Y_LABEL = (np.where(pred > 0.15, 1, 0)) # 이것도 추후 조정해보기!

In [ ]:
submit

In [ ]:
submit.Y_LABEL.value_counts()
# 0.1 445개

In [ ]:
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)